In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd

tree = ET.parse ('../Downloads/padgett.xml')
root = tree.getroot()

Запускаем поиск по вершинам с тегом 'node', чтобы получить список семей.

In [2]:
names = []
for node in root.iter('node'):
    names.append(node.attrib['id'])
names = np.array(names)
n = names.size
print (names)

['ACCIAIUOL' 'ALBIZZI' 'BARBADORI' 'BISCHERI' 'CASTELLAN' 'GINORI'
 'GUADAGNI' 'LAMBERTES' 'MEDICI' 'PAZZI' 'PERUZZI' 'PUCCI' 'RIDOLFI'
 'SALVIATI' 'STROZZI' 'TORNABUON']


В случае со связями нас интересуют только те, что соответствуют брачным связям (PADGM), поэтому и перебирать будем по конкретному поддереву с коренем в root[0][1][0].

In [3]:
root[0][1][0].tag, root[0][1][0].attrib

('network', {'sourceType': 'agent', 'targetType': 'agent', 'id': 'PADGM'})

In [4]:
links = np.empty((n, n))
for link in root[0][1][0].iter('link'):
    links[np.where (names == link.attrib['source'])[0][0]] [np.where (names == link.attrib['target'])[0][0]] = link.attrib['value']
print (links)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0.]]


In [5]:
links.sum (axis=0)

array([1., 3., 2., 3., 3., 1., 4., 1., 6., 1., 3., 0., 3., 2., 4., 3.])

Нулевой строке соответствует семья, не имеющая связей с остальными. Поэтому её авторитет полагае равным 0 и исключаем из рассмотрения.

In [6]:
links = np.delete (links, 11, 0)
links = np.delete (links, 11, 1)
deleted_dynasty = names[11]
names = np.delete (names, 11)
n -= 1
print (names)
print (deleted_dynasty)
print (links)

['ACCIAIUOL' 'ALBIZZI' 'BARBADORI' 'BISCHERI' 'CASTELLAN' 'GINORI'
 'GUADAGNI' 'LAMBERTES' 'MEDICI' 'PAZZI' 'PERUZZI' 'RIDOLFI' 'SALVIATI'
 'STROZZI' 'TORNABUON']
PUCCI
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.]]


Нормируем матрицу связей.

In [7]:
P = np.diag (1/links.sum (axis=0))@links
print (P)

[[0.         0.         0.         0.         0.         0.
  0.         0.         1.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.33333333
  0.33333333 0.         0.33333333 0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.5        0.
  0.         0.         0.5        0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.33333333 0.         0.         0.         0.33333333 0.
  0.         0.33333333 0.        ]
 [0.         0.         0.33333333 0.         0.         0.
  0.         0.         0.         0.         0.33333333 0.
  0.         0.33333333 0.        ]
 [0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.25       0.         0.25       0.        

Итерационно находим стационарное распределение, то есть такую строку $q$, что $q*P = q$.

In [8]:
count = 1000
eps = 1e-9
q = np.ones(n)/n
for i in range (count):
    q_new = q@P
    if (np.linalg.norm (q_new - q) < eps):
        break
    q = q_new
print(q)

[0.025 0.075 0.05  0.075 0.075 0.025 0.1   0.025 0.15  0.025 0.075 0.075
 0.05  0.1   0.075]


Вектор $q$ определяет "авторитеты" каждой из династий.

In [9]:
df = pd.DataFrame ({'Dynasty': names, 'Authority': q})
df = df.append ({'Dynasty': deleted_dynasty, 'Authority': 0}, ignore_index = True)
df.sort_values (by = 'Authority', inplace = True, ascending = False)
df

,Dynasty,Authority
8,MEDICI,0.150
6,GUADAGNI,0.100
13,STROZZI,0.100
4,CASTELLAN,0.075
10,PERUZZI,0.075
11,RIDOLFI,0.075
3,BISCHERI,0.075
14,TORNABUON,0.075
1,ALBIZZI,0.075
2,BARBADORI,0.050
